<a href="https://colab.research.google.com/github/softstat/analysis/blob/main/Angle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd drive

/content/drive


In [4]:
cd MyDrive

/content/drive/MyDrive


In [5]:
cd Data

/content/drive/MyDrive/Data


In [6]:
ls

angle_gait.csv  data.csv  titanic_train.csv  weather.csv


In [94]:
import pandas as pd
angle = pd.read_csv('angle_gait.csv')
clinical_assessment = pd.read_excel('Clinical Assessment.xlsx')

In [19]:
angle.drop('Unnamed: 0',axis=1,inplace=True)

In [142]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

In [55]:
def perform_pca(data, num_components):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    pca = PCA(n_components=num_components)
    pca_result = pca.fit_transform(scaled_data)
    return pca_result

In [58]:
col = angle.columns
col = col.str.replace('[^\w\s]', '', regex=True)
angle.columns = col
num_components = 30

In [60]:
angle = angle.dropna()
na_sum = angle.isna().sum()

In [63]:
import random
random.seed(1)
total_angle_gait_pca = angle.copy()
total_angle_model = PCA(n_components=num_components)
total_angle_result = total_angle_model.fit_transform(total_angle_gait_pca)

In [64]:
total_angle_result

array([[-1.69715644e+02,  2.78757824e+01,  1.24143094e+02, ...,
        -1.45397312e-01,  3.82067471e+00, -3.12725405e+01],
       [-2.23333375e+02, -1.54750144e+01,  4.69437877e+01, ...,
         8.63886626e-01,  2.25162953e+00,  6.28892889e+00],
       [ 6.22312161e+01, -1.43502742e+01,  5.24627212e+02, ...,
        -4.46495142e+00,  1.45622333e+01, -2.38576575e+00],
       ...,
       [ 2.16404983e+02,  2.36416445e+02, -8.90716272e+01, ...,
         1.24347595e+01,  1.59067522e+00,  9.29033228e+00],
       [-1.13801129e+01, -3.60015911e+01, -4.82583542e+01, ...,
         2.09484605e+01,  5.41420545e+00, -1.84546010e+01],
       [ 3.27822467e+01, -1.90162479e+02,  1.36776906e+02, ...,
        -3.80024804e+00, -3.59276959e+01,  1.58570957e+01]])

In [67]:
import numpy as np
angle = np.array(angle)
non_angle = angle[:,0:2424]

In [84]:
random.seed(1)
non_angle_model = PCA(n_components=num_components)
non_angle_result = non_angle_model.fit_transform(non_angle)

In [92]:
non_angle_result.shape

(43, 30)

In [85]:
par_angle = angle[:,2525:4848]
random.seed(1)
par_angle_model = PCA(n_components=num_components)
par_angle_result = par_angle_model.fit_transform(par_angle)

In [128]:
score = clinical_assessment.iloc[:,9]

In [129]:
score = score.dropna()
del score[0]
del score[15]

In [130]:
score

1     43
2     50
3     50
4     51
5     41
6     36
7     51
8     51
9     53
10    41
11    49
12    43
14    44
16    50
17    45
18    19
19    31
20    53
21    52
22    45
23    44
24    44
25    52
26    44
27    49
28    47
29    44
30    48
31    52
32    53
33    45
34    51
35    52
36    33
37    46
38    50
39    52
40    51
41    52
42    51
43    54
44    52
45    43
Name: Berg Balance Scale, dtype: object

In [132]:
y = []
for value in score:
    if value >= 45:
        y.append(1)
    else:
        y.append(0)
print(y)

[0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0]


In [158]:
X = non_angle_result
y = y
X_train , X_test , y_train , y_test  = train_test_split(X,y,test_size = 0.4,random_state=1)

In [161]:
random.seed(1)
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_test)

In [162]:
cm = confusion_matrix(rf_y_pred,y_test)
print(cm)
f1 = f1_score(rf_y_pred,y_test)
print("F1-scoreL",f1)

[[ 0  0]
 [ 3 15]]
F1-scoreL 0.9090909090909091


In [163]:
random.seed(1)
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train)
svm_y_pred = svm_model.predict(X_test)

In [164]:
cm = confusion_matrix(svm_y_pred,y_test)
print(cm)
f1 = f1_score(svm_y_pred,y_test)
print("F1-scoreL",f1)

[[ 2  1]
 [ 1 14]]
F1-scoreL 0.9333333333333333


In [153]:
X = par_angle_result
y = y
X_train , X_test , y_train , y_test  = train_test_split(X,y,test_size = 0.4,random_state=1)

In [154]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_y_pred = rf_model.predict(X_test)

In [155]:
cm = confusion_matrix(rf_y_pred,y_test)
print(cm)
f1 = f1_score(rf_y_pred,y_test)
print("F1-scoreL",f1)

[[ 1  3]
 [ 2 12]]
F1-scoreL 0.8275862068965518


In [ ]:
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_y_pred = xgb_model.predict(X_test)

In [ ]:
lda_model = LDA()
lda_model.fit(X_train, y_train)
lda_y_pred = lda_model.predict(X_test)